In [1]:
!pip install transformers
!pip install pyspark
!pip install sparkdl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=6a36064de66ba230a85ccac5aa952aacce22065465fc78238fb7642b68b3a973
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.d

In [2]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from pyspark.ml import Pipeline
from pyspark.ml.linalg import VectorUDT, Vectors
from transformers import BertModel
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# إعداد جلسة Spark
spark = SparkSession.builder \
    .appName("Text Classification") \
    .getOrCreate()

# تعيين المسارات للمجلدين
fake_news_folder = "/content/drive/MyDrive/dataset/fake"
real_news_folder = "/content/drive/MyDrive/dataset/real"

# قراءة النصوص الكاذبة
texts = []
labels = []
for filename in os.listdir(fake_news_folder):
    file_path = os.path.join(fake_news_folder, filename)
    with open(file_path, "r", encoding='cp1256') as file:
        text = file.read()
        texts.append(text)
        labels.append(0)

# قراءة النصوص الصادقة
for filename in os.listdir(real_news_folder):
    file_path = os.path.join(real_news_folder, filename)
    with open(file_path, "r", encoding='cp1256') as file:
        text = file.read()
        texts.append(text)
        labels.append(1)



In [4]:
# تحميل المحددات من النموذج
tokenizer = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabertv2')

# تحويل النصوص إلى تمثيلات BERT
encoded_texts = []
for news_text in texts:
    inputs = tokenizer.encode_plus(
        news_text,
        None,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )
    encoded_texts.append(inputs['input_ids'])

# تحويل التمثيلات إلى تنسورفلو
features = tf.concat(encoded_texts, axis=0)

# تحويل التسميات إلى تنسورفلو
labels = tf.convert_to_tensor(labels)

# تحويل الأبعاد
features = tf.reshape(features, (-1, 128))

# تحويل features و labels إلى NumPy arrays
features = features.numpy()
labels = labels.numpy()

# خلط البيانات
from sklearn.utils import shuffle

features, labels = shuffle(features, labels, random_state=42)

# تقسيم البيانات إلى بيانات التدريب والاختبار
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.2, random_state=42)

# تحويل البيانات إلى DataFrame في Spark
train_df = spark.createDataFrame(zip(train_features.tolist(), train_labels.tolist()), ["text", "label"])
test_df = spark.createDataFrame(zip(test_features.tolist(), test_labels.tolist()), ["text", "label"])

# تحويل النص إلى نوع Vector
to_vector = udf(lambda a: Vectors.dense(a), VectorUDT())
train_df = train_df.withColumn("text", to_vector("text"))
test_df = test_df.withColumn("text", to_vector("text"))


In [6]:


# إنشاء نموذج CNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(128, 5, activation='relu', input_shape=(128, 1)))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# تحويل DataFrame إلى RDD للتدريب مع TensorFlow
train_rdd = train_df.rdd.map(lambda row: (row['text'].toArray(), int(row['label'])))
train_data = train_rdd.map(lambda x: (x[0].tolist(), x[1])).toDF(['text', 'label'])



In [9]:
# تحويل بيانات التدريب إلى مصفوفات NumPy
train_texts = train_data.select('text').rdd.map(lambda x: x[0]).collect()
train_labels = train_data.select('label').rdd.map(lambda x: x[0]).collect()

# تدريب النموذج
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_texts, train_labels, epochs=100)


Epoch 1/100
51/51 [==============================] - 2s 13ms/step - loss: 81.3910 - accuracy: 0.6200
Epoch 2/100
51/51 [==============================] - 1s 16ms/step - loss: 47.1110 - accuracy: 0.6256
Epoch 3/100
51/51 [==============================] - 1s 15ms/step - loss: 62.8357 - accuracy: 0.6175
Epoch 4/100
51/51 [==============================] - 1s 22ms/step - loss: 63.6768 - accuracy: 0.6287
Epoch 5/100
51/51 [==============================] - 1s 18ms/step - loss: 32.9044 - accuracy: 0.6549
Epoch 6/100
51/51 [==============================] - 1s 22ms/step - loss: 44.0050 - accuracy: 0.6300
Epoch 7/100
51/51 [==============================] - 1s 22ms/step - loss: 65.3717 - accuracy: 0.6095
Epoch 8/100
51/51 [==============================] - 1s 21ms/step - loss: 36.2994 - accuracy: 0.6399
Epoch 9/100
51/51 [==============================] - 1s 22ms/step - loss: 47.0018 - accuracy: 0.6306
Epoch 10/100
51/51 [==============================] - 1s 20ms/step - loss: 35.4221 - accura

In [ ]:

# تحويل DataFrame إلى RDD للتوقعات مع TensorFlow
test_rdd = test_df.rdd.map(lambda row: (row['text'].toArray(), int(row['label'])))
test_data = test_rdd.map(lambda x: (x[0].tolist(), x[1])).toDF(['text', 'label'])

# التوقعات
predictions = model.predict(test_data.select('text').rdd.map(lambda x: x[0]))

# إضافة التوقعات إلى DataFrame
test_df = test_df.withColumn("prediction", predictions)

# تقييم النموذج باستخدام BinaryClassificationEvaluator
binary_evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction")
area_under_roc = binary_evaluator.evaluate(test_df, {binary_evaluator.metricName: "areaUnderROC"})
area_under_pr = binary_evaluator.evaluate(test_df, {binary_evaluator.metricName: "areaUnderPR"})

# تقييم النموذج باستخدام MulticlassClassificationEvaluator
multiclass_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
accuracy = multiclass_evaluator.evaluate(test_df, {multiclass_evaluator.metricName: "accuracy"})
precision = multiclass_evaluator.evaluate(test_df, {multiclass_evaluator.metricName: "weightedPrecision"})
recall = multiclass_evaluator.evaluate(test_df, {multiclass_evaluator.metricName: "weightedRecall"})
f1_score = multiclass_evaluator.evaluate(test_df, {multiclass_evaluator.metricName: "f1"})

# طباعة النتائج
print("Area under ROC:", area_under_roc)
print("Area under PR:", area_under_pr)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)

13/13 [==============================] - 0s 7ms/step


NameError: ignored